## Model Compilation Jupyter Notebook Example

This notebook shows the example of model compilation using edgeai-benchmark.

This script uses TIDL to compile a model and output in a format that edgeai-sdk can understand.

edgeai_benchmark is a python package provided in edgeai-benchmark that provides several functions to assist model compilation.

In [1]:
import os
import tempfile
import argparse
import cv2
import yaml
import shutil
from edgeai_benchmark import *

In [2]:
# the cwd must be the root of the respository
if os.path.split(os.getcwd())[-1] in ('scripts', 'tutorials'):
    os.chdir('../')
#

assert ('TIDL_TOOLS_PATH' in os.environ and 'LD_LIBRARY_PATH' in os.environ and
        'TARGET_SOC' in os.environ), "Check the environment variables"
print("TIDL_TOOLS_PATH=", os.environ['TIDL_TOOLS_PATH'])
print("LD_LIBRARY_PATH=", os.environ['LD_LIBRARY_PATH'])
print("TARGET_SOC=", os.environ['TARGET_SOC'])

print(f"INFO: current dir is: {os.getcwd()}")

TIDL_TOOLS_PATH= /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/tools/AM62A/tidl_tools
LD_LIBRARY_PATH= /home/a0393608local/.pyenv/versions/3.10.14/envs/benchmark/lib/python3.10/site-packages/cv2/../../lib64:/data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/tools/AM62A/tidl_tools:
TARGET_SOC= AM62A
INFO: current dir is: /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark


#### Create a temporary directory. 

This is were the compiled artifacts will be stored.

In [3]:
modelartifacts_tempdir_name = os.path.abspath('./work_dirs_custom')
modelartifacts_custom = os.path.join(modelartifacts_tempdir_name, 'modelartifacts')
print(f'INFO: clearing modelartifacts: {modelartifacts_custom}')
if os.path.exists(modelartifacts_custom):
    shutil.rmtree(modelartifacts_custom)
#

INFO: clearing modelartifacts: /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/work_dirs_custom/modelartifacts


#### Read settings from settings_import_on_pc.yaml

Modify the settings as necessary in the constructor of settings.ConfigSettings()

In [4]:
settings = config_settings.CustomConfigSettings('./settings_import_on_pc.yaml',
                target_device = os.environ['TARGET_SOC'], target_machine = constants.TARGET_MACHINE_PC_EMULATION,
                modelartifacts_path=modelartifacts_custom,
                num_frames=100)

work_dir = os.path.join(settings.modelartifacts_path, f'{settings.tensor_bits}bits')

#### Create Dataset Reader classes

Change the dataset paths according to your dataset location

In [5]:
dataset_calib_cfg = dict(
    path=f'{settings.datasets_path}/imagenetv2c/val',
    split=f'{settings.datasets_path}/imagenetv2c/val.txt',
    num_classes=1000,
    shuffle=True,
    num_frames=min(settings.calibration_frames,50000),
    name='imagenet'
)

# dataset parameters for actual inference
dataset_val_cfg = dict(
    path=f'{settings.datasets_path}/imagenetv2c/val',
    split=f'{settings.datasets_path}/imagenetv2c/val.txt',
    num_classes=1000,
    shuffle=True,
    num_frames=min(settings.num_frames,50000),
    name='imagenet'
)

calib_dataset = datasets.ImageClassification(**dataset_calib_cfg)
val_dataset = datasets.ImageClassification(**dataset_val_cfg)

#### Session runtime_options

The default runtime_options can be overriden by passing a runtime_options dict to this function

In [6]:
# choose one session_name depending on the model type
# tflitert for tflite models, onnxrt for onnx model
#session_name = constants.SESSION_NAME_TFLITERT
session_name = constants.SESSION_NAME_ONNXRT

session_type = settings.get_session_type(session_name)
runtime_options = settings.get_runtime_options(session_name, is_qat=False)

print(session_type)
print(runtime_options)

<class 'edgeai_benchmark.sessions.onnxrt_session.ONNXRTSession'>
{'advanced_options:quantization_scale_type': 4, 'tensor_bits': 8, 'accuracy_level': 1, 'debug_level': 0, 'inference_mode': 0, 'advanced_options:high_resolution_optimization': 0, 'advanced_options:pre_batchnorm_fold': 1, 'advanced_options:calibration_frames': 10, 'advanced_options:calibration_iterations': 10, 'advanced_options:activation_clipping': 1, 'advanced_options:weight_clipping': 1, 'advanced_options:bias_calibration': 1, 'advanced_options:output_feature_16bit_names_list': '', 'advanced_options:params_16bit_names_list': '', 'advanced_options:add_data_convert_ops': 3, 'advanced_options:c7x_firmware_version': '10_01_04_00', 'ti_internal_nc_flag': 83886080}


In [7]:
preproc_transforms = preprocess.PreProcessTransforms(settings)
postproc_transforms = postprocess.PostProcessTransforms(settings)

# these session cfgs also has some default input mean and scale. 
# if your model needs a difference mean and scale, update the session cfg dict being used with those values
onnx_session_cfg = sessions.get_onnx_session_cfg(settings, work_dir=work_dir)
tflite_session_cfg = sessions.get_tflite_session_cfg(settings, work_dir=work_dir)

#### Create pipeline_configs

pipeline_configs is nothing but a dict with the various model configs that we want to compile

In [8]:
pipeline_configs = {
    'cl-mnv2': dict(
        task_type='classification',
        calibration_dataset=calib_dataset,
        input_dataset=val_dataset,
        preprocess=preproc_transforms.get_transform_onnx(),
        session=session_type(**onnx_session_cfg,
            runtime_options=runtime_options,
            model_path=f'{settings.models_path}/vision/classification/imagenet1k/torchvision/mobilenet_v2_tv.onnx'),
        postprocess=postproc_transforms.get_transform_classification(),
        model_info=dict(metric_reference={'accuracy_top1%':71.88})
    ),
}
print(pipeline_configs)

{'cl-mnv2': {'task_type': 'classification', 'calibration_dataset': <edgeai_benchmark.datasets.image_cls.ImageClassification object at 0x79dd83dc34c0>, 'input_dataset': <edgeai_benchmark.datasets.image_cls.ImageClassification object at 0x79dd83dc3550>, 'preprocess': <edgeai_benchmark.preprocess.PreProcessTransforms object at 0x79dd83dc3ca0>, 'session': <edgeai_benchmark.sessions.onnxrt_session.ONNXRTSession object at 0x79dd83dc3c70>, 'postprocess': <edgeai_benchmark.postprocess.PostProcessTransforms object at 0x79dd83dc32b0>, 'model_info': {'metric_reference': {'accuracy_top1%': 71.88}}}}


#### Model Compilation

This will take a few minutes. Please be patient...

In [9]:
# run the model compliation/import and inference
interfaces.run_benchmark_config(settings, work_dir, pipeline_configs)
run_dir = list(pipeline_configs.values())[0]['session'].get_run_dir()
print(f"\nINFO: compiled artifacts is in: {run_dir}")

# print the result:
with open(os.path.join(run_dir, "result.yaml")) as fp:
    result_yaml = yaml.safe_load(fp)
    result_dict = result_yaml['result']
    print(f"INFO: result - {result_dict}")
#

INFO: number of configs: 1


TASKS TOTAL=1, NUM_RUNNING=0:   0%|                                                                                                                                                                                                                                | 0/1 [00:00<?, ?it/s, postfix={'RUNNING': [], 'COMPLETED': []}]


INFO:20250226-142507: starting - cl-mnv2_onnxrt_imagenet1k_torchvision_mobilenet_v2_tv_onnx
INFO:20250226-142507: model_path - /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-modelzoo/models/vision/classification/imagenet1k/torchvision/mobilenet_v2_tv.onnx
INFO:20250226-142507: model_file - /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/work_dirs_custom/modelartifacts/8bits/cl-mnv2_onnxrt_imagenet1k_torchvision_mobilenet_v2_tv_onnx/model/mobilenet_v2_tv.onnx
INFO:20250226-142507: quant_file - /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/work_dirs_custom/modelartifacts/8bits/cl-mnv2_onnxrt_imagenet1k_torchvision_mobilenet_v2_tv_onnx/model/mobilenet_v2_tv_qparams.prototxt
Download done for /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-modelzoo/models/vision/classification/imagenet1k/torchvision/mobilenet_v2_tv.onnx
Download done for /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-modelzoo/models/vision/classification

TASKS TOTAL=1, NUM_RUNNING=1:   0%|                                                                                                                                                                                                                | 0/1 [00:00<?, ?it/s, postfix={'RUNNING': ['cl-mnv2:import'], 'COMPLETED': []}]

Converted model is valid!

INFO:20250226-142507: running - cl-mnv2_onnxrt_imagenet1k_torchvision_mobilenet_v2_tv_onnx

INFO:20250226-142507: pipeline_config - {'task_type': 'classification', 'calibration_dataset': <edgeai_benchmark.datasets.image_cls.ImageClassification object at 0x79dd83dc34c0>, 'input_dataset': <edgeai_benchmark.datasets.image_cls.ImageClassification object at 0x79dd83dc3550>, 'preprocess': <edgeai_benchmark.preprocess.PreProcessTransforms object at 0x79dd83dc3ca0>, 'session': <edgeai_benchmark.sessions.onnxrt_session.ONNXRTSession object at 0x79dd83dc3c70>, 'postprocess': <edgeai_benchmark.postprocess.PostProcessTransforms object at 0x79dd83dc32b0>, 'model_info': {'metric_reference': {'accuracy_top1%': 71.88}}}

INFO:20250226-142507: import  - cl-mnv2_onnxrt_imagenet1k_torchvision_mobilenet_v2_tv_onnx - this may take some time...


...

Traceback (most recent call last):
  File "/data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/edgeai_benchmark/pipelines/pipeline_runner.py", line 264, in _run_pipeline
    result = cls._run_pipeline_impl(settings, pipeline_config, description)
  File "/data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/edgeai_benchmark/pipelines/pipeline_runner.py", line 240, in _run_pipeline_impl
    accuracy_result = accuracy_pipeline(description)
  File "/data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/edgeai_benchmark/pipelines/accuracy_pipeline.py", line 82, in __call__
    param_result = self._run(description=description)
  File "/data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/edgeai_benchmark/pipelines/accuracy_pipeline.py", line 131, in _run
    output_list = self._infer_frames(description)
  File "/data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/edgeai_benchmark/pipelines/accuracy_pipeline.py", line 185, in 

ERROR:20250226-142511: artifacts_folder is missing, please run import (on pc) - /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/work_dirs_custom/modelartifacts/8bits/cl-mnv2_onnxrt_imagenet1k_torchvision_mobilenet_v2_tv_onnx/artifacts


TASKS TOTAL=1, NUM_RUNNING=0: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.43s/it, postfix={'RUNNING': [], 'COMPLETED': ['cl-mnv2']}]



INFO: compiled artifacts is in: /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/work_dirs_custom/modelartifacts/8bits/cl-mnv2_onnxrt_imagenet1k_torchvision_mobilenet_v2_tv_onnx


FileNotFoundError: [Errno 2] No such file or directory: '/data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/work_dirs_custom/modelartifacts/8bits/cl-mnv2_onnxrt_imagenet1k_torchvision_mobilenet_v2_tv_onnx/result.yaml'

#### Package artifacts

Package the artifacts into a .tar.gz file, keeping only the necessary files for inference.

In [ ]:
print()
out_dir = f'{work_dir}_package'
interfaces.package_artifacts(settings, work_dir, out_dir)

#### Download

Download the packaged .tar.gz artifact

TODO: add a download link here, that the user can click to download the packaged artifact .tar.gz file.

In [ ]:
print(f'\nINFO: download the atricats files from the folder: {out_dir}')
print(os.listdir(out_dir))